In [8]:
import numpy as np
import pandas as pd
from scipy.linalg import eig
import os
from docx import Document

In [9]:
# Function to read CSV file
def read_csv(file_path):
    try:
        data = pd.read_csv(file_path)
        print(f"Successfully read {file_path}")
        return data
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

In [10]:
# Function to preprocess the data
def preprocess_data(data):
    # Example preprocessing steps
    data.fillna(0, inplace=True)  # Handle missing values
    return data

In [11]:
# Function to create pairwise comparison matrix
def create_pairwise_matrix(data):
    return np.array(data)

# Function to calculate weights using Eigen vector method
def calculate_weights(matrix):
    eigenvalues, eigenvectors = eig(matrix)
    max_eigenvalue = np.max(eigenvalues)
    eigenvector = eigenvectors[:, np.argmax(eigenvalues)].real
    weights = eigenvector / np.sum(eigenvector)
    return weights, max_eigenvalue

# Function to calculate consistency ratio
def consistency_ratio(matrix, max_eigenvalue, ri):
    n = matrix.shape[0]
    ci = (max_eigenvalue - n) / (n - 1)
    cr = ci / ri
    return ci, cr

In [12]:
# Function to aggregate pairwise matrices
def aggregate_matrices(matrices):
    geo_mean = np.exp(np.mean(np.log(matrices), axis=0))
    return geo_mean


In [13]:
# Function to normalize matrix
def normalize(matrix):
    norm_matrix = matrix / np.sqrt((matrix**2).sum(axis=0))
    return norm_matrix

# Function to calculate weighted normalized matrix
def weighted_normalized_matrix(norm_matrix, weights):
    return norm_matrix * weights

# Function to determine ideal solutions
def ideal_solutions(weighted_matrix):
    positive_ideal = np.max(weighted_matrix, axis=0)
    negative_ideal = np.min(weighted_matrix, axis=0)
    return positive_ideal, negative_ideal

# Function to calculate separation measures
def separation_measures(weighted_matrix, ideal):
    separation = np.sqrt(((weighted_matrix - ideal)**2).sum(axis=1))
    return separation

# Function to calculate closeness ratio
def closeness_ratio(separation_positive, separation_negative):
    return separation_negative / (separation_positive + separation_negative)


In [14]:
def main():
    # Set file paths
    data_folder = './data/'
    result_file = './results/results.docx'

    # Read criteria data
    criteria_file = os.path.join(data_folder, 'criteria.csv')
    criteria_data = read_csv(criteria_file)
    if criteria_data is None:
        print("Failed to read criteria data. Exiting.")
        return

    criteria_matrix = create_pairwise_matrix(criteria_data.values)
    print(f"Criteria matrix shape: {criteria_matrix.shape}")

    if criteria_matrix.shape[0] != criteria_matrix.shape[1]:
        print("Error: Criteria matrix is not square. Exiting.")
        return

    # Calculate weights and consistency for criteria
    weights, max_eigenvalue = calculate_weights(criteria_matrix)
    ri = 1.51  # Correct RI value
    ci, cr = consistency_ratio(criteria_matrix, max_eigenvalue, ri)

    if cr > 0.1:
        print(f"Consistency ratio too high: {cr}. Exiting.")
        return

    # Aggregate pairwise matrices for sub-criteria
    sub_criteria_files = [f for f in os.listdir(data_folder) if f != 'criteria.csv']
    sub_criteria_matrices = []
    for file in sub_criteria_files:
        data = read_csv(os.path.join(data_folder, file))
        if data is not None:
            data = preprocess_data(data)
            matrix = create_pairwise_matrix(data.values)
            print(f"Sub-criteria matrix shape for {file}: {matrix.shape}")

            if matrix.shape[0] != matrix.shape[1]:
                print(f"Error: Sub-criteria matrix in {file} is not square. Skipping.")
                continue

            _, max_eigenvalue = calculate_weights(matrix)
            ci, cr = consistency_ratio(matrix, max_eigenvalue, ri)
            if cr <= 0.1:
                sub_criteria_matrices.append(matrix)
            else:
                print(f"File {file} consistency ratio too high: {cr}")
    
    if not sub_criteria_matrices:
        print("No valid sub-criteria matrices found. Exiting.")
        return

    aggregate_matrix = aggregate_matrices(sub_criteria_matrices)
    agg_weights, _ = calculate_weights(aggregate_matrix)

    # Implement TOPSIS
    # Load alternatives scores dynamically (this is an example, adjust accordingly)
    alternatives = np.random.rand(5, len(weights))  # 5 alternatives and weights length criteria
    decision_matrix = alternatives
    norm_matrix = normalize(decision_matrix)
    weighted_matrix = weighted_normalized_matrix(norm_matrix, agg_weights)
    positive_ideal, negative_ideal = ideal_solutions(weighted_matrix)
    separation_positive = separation_measures(weighted_matrix, positive_ideal)
    separation_negative = separation_measures(weighted_matrix, negative_ideal)
    closeness = closeness_ratio(separation_positive, separation_negative)

    # Rank alternatives
    ranks = np.argsort(closeness)[::-1]

    # Export results to Word document
    doc = Document()
    doc.add_heading('HTA Results', 0)
    doc.add_paragraph(f"Criteria Weights: {weights}")
    doc.add_paragraph(f"Consistency Ratio: {cr}")
    doc.add_paragraph(f"Aggregate Weights: {agg_weights}")
    doc.add_paragraph(f"Closeness Ratios: {closeness}")
    doc.add_paragraph(f"Ranks: {ranks}")
    doc.save(result_file)

if __name__ == "__main__":
    main()

Successfully read ./data/criteria.csv
Criteria matrix shape: (35, 55)
Error: Criteria matrix is not square. Exiting.
